In [1]:
import numpy as np
import os
import tensorflow as tf
import glob
import cv2
from matplotlib import pyplot as plt
import tarfile

In [39]:
def findname(Labels,ids):
    Names=''
    for i in ids:
        for lab in Labels:
            if lab[0]==str(i):
                Names=lab[1]
                break
    return Names
def drawOD(output_dict,img):
    image_np=img
    DBN=int(output_dict['num_detections'])
    for i in range(DBN):
        if output_dict['detection_scores'][i]>0.8:
            boxes=output_dict['detection_boxes'][i]
            clase=findname(MyLabels,[output_dict['detection_classes'][i]])
            h,w,_=image_np.shape            
            cv2.rectangle(image_np, (int(boxes[1]*w), int(boxes[0]*h)), 
                          (int(boxes[3]*w), int(boxes[2]*h)),
                                  (255, 0, 0), 5)

            cv2.putText(image_np,clase, 
                    (int(boxes[1]*w), int(boxes[0]*h)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.8,
                    (255,255,255),
                    2)
    return image_np

def drawmasks(output_dict,img):
    image_np=img.copy()
    DBN=int(output_dict['num_detections'])
    for i in range(DBN):
        if output_dict['detection_scores'][i]>0.8:
            boxes=output_dict['detection_boxes'][i]
            clase=findname(MyLabels,[output_dict['detection_classes'][i]])
            mask=output_dict['detection_masks'][0,i,:,:]
            h,w,_=image_np.shape
            x1,y1,x2,y2=np.int(boxes[1]*w), np.int(boxes[0]*h), np.int(boxes[3]*w), np.int(boxes[2]*h)
            fullm=fullmaskmake([x1,y1,x2,y2],mask)
            bmask=cv2.bitwise_not(fullm*255)
            image_np[y1:y2,x1:x2,0]=cv2.bitwise_and(bmask,image_np[y1:y2,x1:x2,0])
            image_np[y1:y2,x1:x2,1]=cv2.bitwise_and(bmask,image_np[y1:y2,x1:x2,1])
            image_np[y1:y2,x1:x2,2]=cv2.bitwise_and(bmask,image_np[y1:y2,x1:x2,2])
    return image_np

In [29]:

def fullmaskmake(boundingbox,mask):
    x1,y1,x2,y2=boundingbox
    h,w=(y2-y1),(x2-x1)
    fullmask=cv2.resize(mask,(w,h))
    return fullmask    

In [4]:
# What model to download.
MODEL_NAME = 'mask_rcnn_inception_v2_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join(os.getcwd(),'Labels', 'mscoco_label_map.pbtxt')

tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        
#category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
TEST_IMAGE_PATHS=glob.glob('Deteccion/*.jpg')
IMAGE_SIZE = (12, 8)

with open(PATH_TO_LABELS, 'r') as fh:
    graph_str = fh.read()
    
    
Labelsf=graph_str.split('item')
MyLabels=[]
for index in Labelsf:
    pos1id=index.find('id: ')+4
    pos2id=pos1id+2
    if not index[pos1id:pos2id].isdigit():
        pos2id-=1
    pos1dn=index.find('display_name: ')+15
    pos2dn=index.find('"',pos1dn)
    MyLabels.append([index[pos1id:pos2id],index[pos1dn:pos2dn]])

In [6]:
def run_inference_for_single_image(image, sess):
# Get handles to input and output tensors
    ops = tf.get_default_graph().get_operations()
    all_tensor_names = {output.name for op in ops for output in op.outputs}
    tensor_dict = {}
    for key in [
      'num_detections', 'detection_boxes', 'detection_scores',
      'detection_classes', 'detection_masks'
  ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
            tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        #detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
        #    detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)        
    
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
  # Run inference
    output_dict = sess.run(tensor_dict,
                         feed_dict={image_tensor: np.expand_dims(image, 0)})
  # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
              'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    return output_dict



In [9]:
sess=tf.InteractiveSession(graph=detection_graph)

In [38]:
TEST_IMAGE_PATHS=glob.glob(os.path.join(os.getcwd(),'images','*'))
for image_path in TEST_IMAGE_PATHS:
    image_np = cv2.imread(image_path)
    output_dict = run_inference_for_single_image(image_np, sess)
    image_np=drawOD(output_dict,image_np)
    image_np=drawmasks(output_dict,image_np)
    image_np=cv2.resize(image_np,(1280,720))
    cv2.imshow('j',image_np)
    cv2.waitKey(3000)
cv2.destroyAllWindows()

In [40]:
cam = cv2.VideoCapture(0)
while(1):
    ret,frame=cam.read()
    output_dict = run_inference_for_single_image(frame, sess)
    frame=drawOD(output_dict,frame)
    frame=drawmasks(output_dict,frame)
    frame=cv2.resize(frame,(1280,720))
    cv2.imshow('j',frame)
    k=cv2.waitKey(10) & 0xFF
    if k==ord('q'):
        break
cv2.destroyAllWindows()

In [17]:
sess.close()